In [1]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\carlo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Configuración Selenium
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = "https://www.turismo.gal/localizador-de-recursos?langId=es_ES"
driver.get(url)

print("✅ Selecciona manualmente los filtros que quieras (municipio, interés, etc.)")
print("⏳ Tienes 30 segundos para hacerlo antes de que empiece la extracción...")

time.sleep(30)

wait = WebDriverWait(driver, 20)
data = []
pagina = 1

while True:
    print(f"📄 Extrayendo página {pagina}...")

    # Esperar resultados
    resultados = wait.until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.clearfix.list-item"))
    )

    for r in resultados:
        try:
            nombre_elem = r.find_element(By.CSS_SELECTOR, "a.title.mod-href")
            nombre = nombre_elem.text.strip()
            link = nombre_elem.get_attribute("href")  # enlace del nombre
        except:
            nombre = None
            link = None

        try:
            direccion = r.find_element(By.CSS_SELECTOR, "div.col-sm-12.address .media-body").text.strip()
        except:
            direccion = None

        try:
            detalle_raw = r.find_element(By.CSS_SELECTOR, "small.show").text.strip()
            if "-" in detalle_raw:
                partes = detalle_raw.split("-", 1)
                tipo = partes[0].strip()
                categoria = partes[1].strip()
            else:
                tipo = detalle_raw.strip()
                categoria = None
        except:
            detalle_raw = None
            tipo = None
            categoria = None

        try:
            telefono = r.find_element(By.CSS_SELECTOR, "div.telefono").text.strip()
        except:
            telefono = None

        try:
            email_elem = r.find_element(By.CSS_SELECTOR, "a[href^='mailto:']")
            email = email_elem.text.strip()
        except:
            email = None

        data.append({
            "nombre": nombre,
            "link": link,
            "direccion": direccion,
            "tipo": tipo,
            "categoria": categoria,
            "telefono": telefono,
            "email": email
        })

    # Verificar si el botón siguiente está deshabilitado
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.btn.btn-default.btn-pagination[title='Siguiente']")
        if "disabled" in next_button.get_attribute("class"):
            print("🚩 Última página alcanzada.")
            break
        else:
            driver.execute_script("arguments[0].click();", next_button)
            pagina += 1
            time.sleep(3)
    except:
        print("🚩 No se encontró botón siguiente. Fin.")
        break

# Guardar CSV
df = pd.DataFrame(data)
df.to_csv("ServiciosTurisiticos.csv", index=False, encoding="utf-8-sig")

print("✅ Datos guardados en ServiciosTurisiticos.csv")
driver.quit()


✅ Selecciona manualmente los filtros que quieras (municipio, interés, etc.)
⏳ Tienes 30 segundos para hacerlo antes de que empiece la extracción...
📄 Extrayendo página 1...
📄 Extrayendo página 2...
📄 Extrayendo página 3...
📄 Extrayendo página 4...
📄 Extrayendo página 5...
📄 Extrayendo página 6...
📄 Extrayendo página 7...
📄 Extrayendo página 8...
📄 Extrayendo página 9...
📄 Extrayendo página 10...
📄 Extrayendo página 11...
📄 Extrayendo página 12...
📄 Extrayendo página 13...
📄 Extrayendo página 14...
📄 Extrayendo página 15...
📄 Extrayendo página 16...
📄 Extrayendo página 17...
📄 Extrayendo página 18...
📄 Extrayendo página 19...
📄 Extrayendo página 20...
📄 Extrayendo página 21...
📄 Extrayendo página 22...
📄 Extrayendo página 23...
📄 Extrayendo página 24...
📄 Extrayendo página 25...
📄 Extrayendo página 26...
🚩 Última página alcanzada.
✅ Datos guardados en ServiciosTurisiticos.csv


In [16]:
import pandas as pd
import glob
import os

# Carpeta donde tienes los CSV
carpeta = "datos/"  # cambia si están en otra ruta

# Buscar todos los CSV en la carpeta
archivos = glob.glob(os.path.join(carpeta, "*.csv"))

print(f"🔎 Encontrados {len(archivos)} archivos CSV")

# Leer y concatenar
dfs = []
for archivo in archivos:
    print(f"📂 Leyendo {archivo}...")
    df = pd.read_csv(archivo)

     # Crear columna "fuente" con el nombre del archivo sin la extensión .csv
    df["fuente"] = os.path.splitext(os.path.basename(archivo))[0]
    dfs.append(df)

# Unir todo en un solo DataFrame
df_final = pd.concat(dfs, ignore_index=True)

# Eliminar duplicados (opcional)
#df_final.drop_duplicates(inplace=True)

# Guardar en un único CSV
df_final.to_csv("resultados_unidos.csv", index=False, encoding="utf-8-sig")

print("✅ Archivos unidos en resultados_unidos.csv")

🔎 Encontrados 7 archivos CSV
📂 Leyendo datos\Donde alojarse.csv...
📂 Leyendo datos\Donde comer.csv...
📂 Leyendo datos\Donde informarse.csv...
📂 Leyendo datos\Que Hacer.csv...
📂 Leyendo datos\Que visitar.csv...
📂 Leyendo datos\Servicios.csv...
📂 Leyendo datos\Turismo transporte.csv...
✅ Archivos unidos en resultados_unidos.csv


In [5]:
from selenium import webdriver  
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os

# === 1. Configuración Selenium ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 15)

# === 2. Leer CSV existente ===
csv_file = r"datos/Servicios.csv"
df = pd.read_csv(csv_file, encoding="utf-8-sig")

# Añadimos columnas fijas si no existen
for col in ["coordenadas", "web"]:
    if col not in df.columns:
        df[col] = None

# === 3. Comprobar progreso previo ===
progreso_file = "progreso_Turismo_transporte.txt"
if os.path.exists(progreso_file):
    with open(progreso_file, "r") as f:
        last_index = int(f.read().strip())
    print(f"♻️ Retomando desde la fila {last_index}")
else:
    last_index = 0

# === 4. Recorrer desde la última fila procesada ===
for i, row in df.iloc[last_index:].iterrows():
    url = row["link"]
    print(f"🌐 Procesando fila {i} -> {row['nombre']} -> {url}")

    try:
        driver.get(url)
        time.sleep(2)

        # --- Extraer coordenadas ---
        try:
            coords_elem = wait.until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='row padding-top-s']"))
            )
            coordenadas = coords_elem.text.replace("Coordenadas:", "").strip()
        except:
            coordenadas = None

        # --- Extraer web oficial ---
        try:
            web_elems = driver.find_elements(By.XPATH, "//div[@class='media-body cont-info-contact']/a")
            web = None
            for elem in web_elems:
                href = elem.get_attribute("href")
                if href and href.startswith("http"):
                    web = href
                    break  # toma solo el primero que cumpla la condición
        except Exception as e:
            print(f"⚠️ Error extrayendo web en fila {i}: {e}")
            web = None


        # --- Extraer info-especial ---
        bloques_info = driver.find_elements(By.CLASS_NAME, "info-especial")
        print(f"🔎 Se encontraron {len(bloques_info)} bloques info-especial en esta página")

        for bloque in bloques_info:
            grupos = bloque.find_elements(By.CLASS_NAME, "bg-group") + \
                     bloque.find_elements(By.CLASS_NAME, "grupo.desplegador.fila")

            for g in grupos:
                texto = g.text.strip()
                if not texto:
                    continue

                lineas = texto.split("\n")
                titulo = lineas[0].strip()
                col_name = titulo.lower().replace(" ", "_")

                if col_name not in df.columns:
                    df[col_name] = None

                if "Número de cuartos" not in texto:
                    contenido = " | ".join(lineas[1:]).strip()
                    df.at[i, col_name] = contenido
                else:
                    filas = lineas[2:]
                    contenido_limpio = []
                    for fila in filas:
                        partes = fila.split()
                        if len(partes) < 3:
                            continue
                        habitacion = " ".join(partes[:-2])
                        cuartos = partes[-2]
                        plazas = partes[-1]
                        registro = f"Habitación: {habitacion} | Número de cuartos: {cuartos} | Número de plazas: {plazas}"
                        contenido_limpio.append(registro)
                    df.at[i, col_name] = "; ".join(contenido_limpio)

        # --- Extraer situacion-acceso ---
        bloques_situacion = driver.find_elements(By.CLASS_NAME, "situacion-acceso")
        print(f"🔎 Se encontraron {len(bloques_situacion)} bloques situacion-acceso en esta página")

        for bloque in bloques_situacion:
            grupos = bloque.find_elements(By.CLASS_NAME, "bg-group")
            for g in grupos:
                texto = g.text.strip()
                if not texto:
                    continue

                lineas = texto.split("\n")
                titulo = lineas[0].strip()
                col_name = "situacion_" + titulo.lower().replace(" ", "_")

                if col_name not in df.columns:
                    df[col_name] = None

                contenido = " | ".join(lineas[1:]).strip()
                df.at[i, col_name] = contenido

        # --- Guardar las columnas base ---
        df.at[i, "coordenadas"] = coordenadas
        df.at[i, "web"] = web

        # === Guardar progreso y CSV ---
        df.to_csv("Datos_Turismo_transporte1.csv", index=False, encoding="utf-8-sig")
        with open(progreso_file, "w") as f:
            f.write(str(i + 1))  # +1 porque ya procesamos esta fila

    except Exception as e:
        print(f"⚠️ Error en fila {i} -> {url}: {e}")
        # Guardar progreso aún si hay error
        with open(progreso_file, "w") as f:
            f.write(str(i))
        continue

driver.quit()
print("✅ Proceso completado. CSV actualizado y progreso guardado.")


♻️ Retomando desde la fila 0
🌐 Procesando fila 0 -> ABRAMAR VIAJES -> https://www.turismo.gal/recurso/-/detalle/4554/abramar-viajes-?langId=es_ES&tp=73&ctre=153
🔎 Se encontraron 3 bloques info-especial en esta página
🔎 Se encontraron 1 bloques situacion-acceso en esta página
🌐 Procesando fila 1 -> ADEMAR VIAJES -> https://www.turismo.gal/recurso/-/detalle/33594/ademar-viajes?langId=es_ES&tp=73&ctre=153
🔎 Se encontraron 3 bloques info-especial en esta página
🔎 Se encontraron 1 bloques situacion-acceso en esta página
🌐 Procesando fila 2 -> AGENCIA MAS VIAJES -> https://www.turismo.gal/recurso/-/detalle/230310000215/agencia-mas-viajes?langId=es_ES&tp=73&ctre=153
🔎 Se encontraron 3 bloques info-especial en esta página
🔎 Se encontraron 1 bloques situacion-acceso en esta página
🌐 Procesando fila 3 -> AGENCIA SANVITUR -> https://www.turismo.gal/recurso/-/detalle/220103000002/agencia-sanvitur?langId=es_ES&tp=73&ctre=153
🔎 Se encontraron 3 bloques info-especial en esta página
🔎 Se encontraron 1

In [ ]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
import urllib.parse
import re
import random

# === 1. Configuración Selenium ===
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
# options.add_argument("--headless")  # Descomenta si quieres ejecutar sin abrir la ventana

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

# === 2. Leer CSV existente ===
csv_file = r"datos_pontevedra_Municipio_limpio.csv"
df = pd.read_csv(csv_file, encoding="utf-8-sig", sep=';')

# Añadir columnas si no existen
for col in [
    "nombre_tripadvisor",
    "link_tripadvisor",
    "calificacion_tripadvisor",
    "ubicacion_tripadvisor",
    "web_tripadvisor",
    "telefono_tripadvisor",
    "correo_tripadvisor"
]:
    if col not in df.columns:
        df[col] = None

# === 3. Progreso previo ===
progreso_file = "progreso_tripadvisor_nombre.txt"
if os.path.exists(progreso_file):
    with open(progreso_file, "r") as f:
        last_index = int(f.read().strip())
    print(f"♻️ Retomando desde la fila {last_index}")
else:
    last_index = 0


# === Funciones auxiliares ===
def parse_numeric(s):
    if not s:
        return None
    s = s.replace(",", ".")
    m = re.search(r"(\d+(?:\.\d+)?)", s)
    return m.group(1) if m else None


def extract_rating(driver):
    xpaths_try = [
        "//div[contains(@class,'VVbkp')]//span[@aria-hidden='true']",
        "//div[contains(@class,'MyMKp') or contains(@class,'MyMKp u')]//span[@aria-hidden='true']",
        "//span[contains(@aria-label,'of 5') or contains(@aria-label,'de 5')]",
        "//span[contains(@class,'bubble_')]",
        "//img[contains(@alt,'of 5') or contains(@alt,'de 5')]",
        "//h1//following::span[@aria-hidden='true'][1]",
        "//h1//following::span[contains(text(),'.') or contains(text(),',')][1]",
    ]

    for xp in xpaths_try:
        elems = driver.find_elements(By.XPATH, xp)
        if not elems:
            continue
        for e in elems:
            text = (e.get_attribute("textContent") or e.text or "").strip()
            if text:
                val = parse_numeric(text)
                if val:
                    return val
            aria = (e.get_attribute("aria-label") or "").strip()
            if aria:
                val = parse_numeric(aria)
                if val:
                    return val
            alt = (e.get_attribute("alt") or "").strip()
            if alt:
                val = parse_numeric(alt)
                if val:
                    return val
            cls = (e.get_attribute("class") or "")
            m = re.search(r"bubble_(\d+)", cls)
            if m:
                return str(int(m.group(1)) / 10)
    return None


def extract_contact_info(driver):
    """Extrae web, teléfono y correo desde el bloque de contactos."""
    web, tel, correo = None, None, None
    try:
        links = driver.find_elements(
            By.XPATH,
            "//div[contains(@class,'CwizA')]/a[contains(@class,'xyffN')]"
        )
        for link in links:
            href = link.get_attribute("href") or ""
            if href.startswith("tel:"):
                tel = href.replace("tel:", "").strip()
            elif href.startswith("mailto:"):
                correo = href.replace("mailto:", "").strip()
            elif href.startswith("http"):
                web = href.strip()
    except Exception:
        pass
    return web, tel, correo


def extract_location(driver):
    """
    Extrae la ubicación de la página usando las clases y estructuras específicas proporcionadas.
    """
    posibles_xpaths = [
        # Buscar ubicación dentro de los divs sin span
        "//div[contains(@class,'vcYDc P e H o')]//div[contains(@class,'biGQs _P ezezH')]",
        # Otra opción de búsqueda para la dirección
        "//*[contains(@data-automation,'restaurantsMapLinkOnName')]//span",  # Ya tenías esta búsqueda
    ]

    for xp in posibles_xpaths:
        elems = driver.find_elements(By.XPATH, xp)
        for e in elems:
            texto = (e.text or "").strip()
            if texto and len(texto) > 5:
                # Filtro de palabras clave que no son direcciones
                if not any(bad in texto.lower() for bad in [
                    "opinión", "reseña", "comentario", "ver más", "página web", "reviews", "review"
                ]):
                    print(f"[DEBUG] Dirección encontrada: {texto}")
                    return texto

    return None

def extract_about(driver):
    """
    Extrae el texto del "Acerca de" desde la clase 'NbcSk e'.
    """
    try:
        about_elem = driver.find_element(By.XPATH, "//div[contains(@class,'NbcSk e')]")
        return about_elem.text.strip() if about_elem else None
    except Exception as e:
        print(f"⚠️ Error extrayendo el 'Acerca de': {e}")
        return None

def scroll_to_bottom(driver, max_scrolls=20, wait_time=2):
    """Desplaza la página hacia abajo repetidamente para cargar todo el contenido dinámico."""
    last_height = driver.execute_script("return document.body.scrollHeight")  # Obtén la altura inicial

    for _ in range(max_scrolls):
        # Desplázate hasta el fondo
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)  # Espera para que el contenido cargue

        # Compara la altura actual con la anterior
        new_height = driver.execute_script("return document.body.scrollHeight")

        # Si la altura no cambia, significa que ya hemos llegado al final
        if new_height == last_height:
            break
        
        last_height = new_height  # Actualiza la altura para el siguiente desplazamiento

# === 4. Recorrer filas ===
for i, row in df.iloc[last_index:].iterrows():
    nombre = str(row["nombre"]).strip() if "nombre" in df.columns else None
    if not nombre or pd.isna(nombre):
        continue

    print(f"\n🌍 Buscando en TripAdvisor: {nombre}")

    try:
        ubicacion = "Pontevedra Galicia España"
        query = urllib.parse.quote(f"{nombre} {ubicacion}")
        search_url = f"https://www.tripadvisor.com/Search?q={query}"
        driver.get(search_url)
        time.sleep(random.uniform(8, 12))  # búsqueda más rápida

        # Buscar primer resultado válido
        try:
            primer_link = wait.until(
                EC.presence_of_element_located(
                    (By.XPATH, "(//a[contains(@href,'/Hotel_') or contains(@href,'/Attraction_') or contains(@href,'/Restaurant_')])[1]")
                )
            )
            href = primer_link.get_attribute("href")
        except:
            print(f"⚠️ No se encontró resultado válido para '{nombre}'")
            df.at[i, "nombre_tripadvisor"] = "No encontrado"
            df.to_csv("Datos_Turismo_tripadvisor_nombres.csv", index=False, encoding="utf-8-sig", sep=';')
            with open(progreso_file, "w") as f:
                f.write(str(i + 1))
            continue

        # Entrar al resultado
        driver.get(href)
        time.sleep(random.uniform(10, 20))  # espera más corta, máximo 20s dentro del perfil

        # Desplazar hacia abajo para cargar contenido dinámico
        scroll_to_bottom(driver, max_scrolls=30, wait_time=3)  # Desplazamiento con tiempo de espera

        # Extraer nombre principal
        try:
            nombre_elem = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ReGgO")))
            nombre_tripadvisor = nombre_elem.text.strip()
        except:
            nombre_tripadvisor = None

        # Extraer calificación
        calificacion = extract_rating(driver)

        # Extraer ubicación mejorada
        ubicacion_texto = extract_location(driver)

        # Extraer contacto (web, teléfono, correo)
        web, tel, correo = extract_contact_info(driver)

        # Extraer 'Acerca de' desde la clase NbcSk e
        acerca_de = extract_about(driver)

        # Guardar resultados
        df.at[i, "nombre_tripadvisor"] = nombre_tripadvisor
        df.at[i, "link_tripadvisor"] = href
        df.at[i, "calificacion_tripadvisor"] = calificacion
        df.at[i, "ubicacion_tripadvisor"] = ubicacion_texto
        df.at[i, "web_tripadvisor"] = web
        df.at[i, "telefono_tripadvisor"] = tel
        df.at[i, "correo_tripadvisor"] = correo
        df.at[i, "acerca_de_tripadvisor"] = acerca_de  # Añadir esta columna

        print(f"✅ Nombre: {nombre_tripadvisor if nombre_tripadvisor else 'No encontrado'} | ⭐ {calificacion if calificacion else 'No disponible'}")
        print(f"📍 {ubicacion_texto if ubicacion_texto else 'Sin dirección'} | 🌐 {web if web else 'Sin web'} | 📞 {tel if tel else 'Sin teléfono'} | ✉️ {correo if correo else 'Sin correo'}")
        print(f"📝 Acerca de: {acerca_de if acerca_de else 'No disponible'}")

        # Guardar CSV y progreso
        df.to_csv("Datos_Turismo_tripadvisor_nombres.csv", index=False, encoding="utf-8-sig", sep=';')
        with open(progreso_file, "w") as f:
            f.write(str(i + 1))

        # Espera entre búsquedas (máximo 40s total)
        pausa = random.uniform(10, 25)
        print(f"🕒 Esperando {pausa:.1f} segundos antes de la siguiente búsqueda...")
        time.sleep(pausa)

        if (i + 1) % 10 == 0:
            descanso = random.uniform(40, 60)
            print(f"😴 Descanso largo de {descanso:.0f} segundos para evitar bloqueos...")
            time.sleep(descanso)

    except Exception as e:
        print(f"⚠️ Error procesando '{nombre}': {e}")
        with open(progreso_file, "w") as f:
            f.write(str(i))
        continue

driver.quit()
print("\n✅ Proceso completado. CSV actualizado y progreso guardado.")
